In [1]:
import numpy as np
from scipy.stats import unitary_group
from collections import deque
import random
import scipy as sc
import sys
sys.path.insert(0, '/home/aditya/Desktop/Research_PhD/chflow/src/')
from define import randchans as rc
from define import qcode as qc
from define import globalvars as gv

In [2]:
def extend_gate(support, mat, extended_support):
    r"""
    Extend a gate supported on some qubits in support, to a set of qubit labels, by padding identities.

    Let the given gate :math:`G` have support on

    .. math::
        :nowrap:

        \begin{gather*}
        \vec{s} = s_{1} s_{2} \ldots s_{m}
        \end{gather*}

    and the extended support be

    .. math::
        :nowrap:

        \begin{gather*}
        \vec{e} = e_{1} e_{2} \ldots e_{N}
        \end{gather*}

    - Let :math:`\Gamma \gets \mathbb{I}`

    - For each :math:`i` in 1 to :math:`m`

            - Let :math:`j~\gets` index of :math:`s_{i}` of :math:`\vec{e}`

            - For :math:`k = j-1` to :math:`i`, do

            - Let :math:`d \gets i - (j-1)`

            - Let :math:`\Gamma \gets \Gamma \cdot SWAP_{k, k+d}.`

    - return

    .. math::
        :nowrap:

        \begin{gather*}
        \Gamma \cdot (G \otimes \mathbb{I}^{\otimes (N - m)}) \cdot \Gamma^{-1}
        \end{gather*}
    """
    SWAP = np.array(
                    [[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]], dtype=np.complex128
                )
#     print("Extend {} on {} to {}".format(mat, support, extended_support))
    support_range = np.array(
        [np.asscalar(np.argwhere(extended_support == s)) for s in support], dtype=np.int
    )
    # print("support_range = {}".format(support_range))
    if np.asscalar(np.ptp(support_range)) < 2:
        # print(
        #     "M = I_(%d) o G o I_(%d)"
        #     % (np.min(support_range), extended_support.size - 1 - np.max(support_range))
        # )
        return Kron(
            np.identity(2 ** np.min(support_range)),
            dot(SWAP, mat, SWAP.T) if (support_range[0] > support_range[-1]) else mat,
            np.identity(2 ** (extended_support.size - 1 - np.max(support_range))),
        )
    swap = np.identity(2 ** extended_support.size, dtype=np.double)
    for i in range(len(support)):
        j = np.asscalar(np.argwhere(extended_support == support[i]))
        d = np.sign(i - j)
        # print("bring {} to {} in direction {}".format(j, i, d))
        if not (d == 0):
            for k in range(j, i, d):
                swap = dot(
                    swap,
                    extend_gate(
                        [extended_support[k], extended_support[k + d]],
                        SWAP,
                        extended_support,
                    ),
                )
    # print("M = G o I_(%d)" % (extended_support.size - len(support)))
    return dot(
        swap,
        Kron(mat, np.identity(2 ** (extended_support.size - len(support)))),
        swap.T,
    )


In [3]:
def Dagger(x):
    return np.conj(np.transpose(x))
def Kron(*mats):
    """
    Extend the standard Kronecker product, to a list of matrices, where the Kronecker product is recursively taken from left to right.
    """
    if len(mats) < 2:
        return mats[0]
    return np.kron(mats[0], Kron(*(mats[1:])))

def Dot(*mats):
    """
    Extend the standard Dot product, to a list of matrices, where the Kronecker product is recursively taken from left to right.
    """
    if len(mats) < 2:
        return mats[0]
    return np.dot(mats[0], dot(*(mats[1:])))
def circular_shift(li,start,end,direction="right"):
    r"""
    Circular shifts a part of the list `li` between start and end indices
    """
    d = deque(li[start:end+1])
    if(direction=="right"):
        d.rotate(1)
    else:
        d.rotate(-1)
    return li[:start] + list(d) + li[end+1:]

def get_Pauli_tensor(Pauli):
    r"""
    Takes an n-qubit pauli as a list and converts it to the tensor form
    """
    listOfPaulis = [gv.Pauli[i] for i in Pauli]
    if listOfPaulis:
        Pj = listOfPaulis[0]
        n_qubits = len(listOfPaulis)
        for i in range(1,n_qubits):
            Pj = np.tensordot(Pj,listOfPaulis[i],axes=0)
        indices_Pj = [i for i in range(0,len(Pj.shape),2)] + [i for i in range(1,len(Pj.shape),2)]
    else:
        raise ValueError(
            f"Invalid Pauli {Pauli} "
        )
    return np.transpose(Pj,indices_Pj)
    


In [4]:
def fix_index_after_tensor(tensor,indices_changed):
    r"""
    Tensor product alters the order of indices. This function helps reorder to fix them back. 
    """
    n = len(tensor.shape)-1
    perm_list = list(range(len(tensor.shape)))
    n_changed = len(indices_changed)
    for i in range(len(indices_changed)):
        index = indices_changed[i]
        perm_list = circular_shift(perm_list,index,n-n_changed+i+1,"right")
    return(np.transpose(tensor,perm_list))

    

In [5]:
def get_PTMelem_ij(krausdict,Pi,Pjlist,n_qubits):
    r"""
    Assumes Paulis Pi,Pj to be a tensor on n_qubits
    Calculates Tr(Pj Eps(Pi)) for each Pj in Pjlist
    Kraus dict has the format ("support": list of kraus ops on the support)
    Assumes qubits
    Assumes kraus ops to be square with dim 2**(number of qubits in support)
    """
#     Pres stores the addition of all kraus applications
#     Pi_term stores result of individual kraus applications to Pi
    Pres = np.zeros_like(Pi)
    for key,(support,krausList) in krausdict.items():
        indices = support + tuple(map(lambda x:x+n_qubits,support))
        for kraus in krausList:
            kraus_reshape_dims = [2]*(2*int(np.log2(kraus.shape[0])))
            indices_Pi = indices[len(indices)//2:] 
            indices_kraus = range(len(kraus_reshape_dims)//2)
            Pi_term = np.tensordot(Pi,Dagger(kraus).reshape(kraus_reshape_dims),(indices_Pi,indices_kraus))
            Pi_term = fix_index_after_tensor(Pi_term,indices_Pi)
            indices_Pi = indices[:len(indices)//2]
            indices_kraus = range(len(kraus_reshape_dims))[len(kraus_reshape_dims)//2:]
            Pi_term = np.tensordot(Pi_term,kraus.reshape(kraus_reshape_dims),(indices_Pi,indices_kraus))
            Pi_term = fix_index_after_tensor(Pi_term,indices_Pi)
            Pres += Pi_term
    # take dot product with Pj and trace
    trace_vals = np.zeros(len(Pjlist),dtype=np.double)
    indices_Pi = list(range(len(Pi.shape)//2))
    indices_Pj = list(range(len(Pi.shape)//2,len(Pi.shape)))
    for i in range(len(Pjlist)):
        Pj = Pjlist[i]
        Pres_times_Pj = np.tensordot(Pres,Pj,(indices_Pi,indices_Pj))
        # Take trace
        einsum_inds = list(range(len(Pres_times_Pj.shape)//2))+list(range(len(Pres_times_Pj.shape)//2))
        trace_vals[i] = np.real(np.einsum(Pres_times_Pj,einsum_inds))/2**n_qubits
    return trace_vals
    
    

In [6]:
def get_kraus_unitaries(p_error,rotation_angle,qcode):
    r"""
    Sub-routine to prepare the dictionary for error eps = sum of unitary errors
    Generates kraus as random multi-qubit unitary operators rotated by rotation_angle
    Probability associated with each weight-k error scales exponentially p_error^k (except weight 2)
    Returns :
    dict[key] = (support,krauslist)
    where key = number associated to the operation applied (not significant)
    support = tuple describing which qubits the kraus ops act on
    krauslist = krauss ops acting on support    
    """
    total_probs = 0
    kraus_count = 0
    kraus_dict = {}
    for n_q in range(1,qcode.N+1):
        if(n_q==1):
            p_q = p_error
        elif(n_q==2):
            p_q = 0.1*p_error
        else:
            p_q = np.power(p_error,n_q)
#         Number of unitaries of weight n_q is max(1,qcode.N-n_q-1)
        for __ in range(max(1,qcode.N-n_q-1)):
            support = tuple(sorted((random.sample(range(qcode.N),n_q))))
            rand_unitary  = rc.RandomUnitary(rotation_angle, 2**n_q, method="exp")
            kraus_dict[kraus_count] = (support,[rand_unitary*np.sqrt(p_q)])
            total_probs += p_q
            kraus_count += 1 
    #     Renormalize kraus ops
    for key,(support,krauslist) in kraus_dict.items():
        for k in range(len(krauslist)):
            kraus_dict[key][1][k] /= np.sqrt(total_probs)
    return kraus_dict  
            
        

In [16]:
def get_process_LS(p_error,rotation_angle,qcode):
    r"""
    Generates LS part of the process matrix for Eps = sum of unitary errors
    p_error^k is the probability associated to a k-qubit unitary (except weight 2)
    rotation_angle is the angle used for each U = exp(i*rotation_agnle*H)
    return linearized process matrix in LS ordering for T=0
    """
    nstabs = 2 ** (qcode.N - qcode.K)
    nlogs = 4 ** qcode.K
    ops = qc.GetOperatorsForTLSIndex(qcode,range(nstabs*nlogs))
    ops_tensor = list(map(get_Pauli_tensor,ops))
    kraus_dict = get_kraus_unitaries(p_error,rotation_angle,qcode)
    process = np.zeros(nstabs*nstabs*nlogs*nlogs,dtype=np.double)
    for i in range(len(ops_tensor)):
        process[i*nstabs*nlogs:(i+1)*nstabs*nlogs] = get_PTMelem_ij(kraus_dict,ops_tensor[i],ops_tensor,qcode.N)
    return process

def get_process_diagLST(p_error,rotation_angle,qcode):
    r"""
    Generates diagonal of the process matrix in LST ordering for Eps = sum of unitary errors
    p_error^k is the probability associated to a k-qubit unitary (except weight 2)
    rotation_angle is the angle used for each U = exp(i*rotation_agnle*H)
    return linearized diagonal of the process matrix in LST ordering 
    """
    nstabs = 2 ** (qcode.N - qcode.K)
    nlogs = 4 ** qcode.K
    kraus_dict = get_kraus_unitaries(p_error,rotation_angle,qcode)
    diag_process = np.zeros(nstabs*nstabs*nlogs,dtype=np.double)
    ops = qc.GetOperatorsForLSTIndex(qcode,range(nstabs*nstabs*nlogs))
    for i in range(len(diag_process)):
        op_tensor = get_Pauli_tensor(ops[i])
        diag_process[i] = get_PTMelem_ij(kraus_dict,op_tensor,[op_tensor],qcode.N)[0]
    return diag_process
    
    
    

In [8]:
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)

In [17]:
%%time
process = get_process_diagLST(0.2,0.05,qcode)

CPU times: user 7min 27s, sys: 13.3 s, total: 7min 40s
Wall time: 3min 52s


In [13]:
process

array([1.        , 0.99878526, 0.99738266, ..., 0.99636942, 0.99483617,
       0.9946984 ])

In [ ]:
# def random_input_PTelem_ij(dkraus,nkraus,jlistlength,n_qubits = 5):
#     pauli_list = [np.eye(2),np.array([[0,1],[1,0]]),np.array([[0,-1j],[1j,0]]),np.array([[1,0],[0,-1]])]
#     Pi = get_Pauli_tensor([random.choice(pauli_list)]*n_qubits)
#     Pjlist = []
#     for i in range(jlistlength):
#         Pjlist.append(get_Pauli_tensor([random.choice(pauli_list)]*n_qubits))
#     krausdict = {}
#     for i in range(nkraus):
#         k = dkraus
#         support = tuple(sorted((random.sample(range(n_qubits),k))))
# #         print(support)
#         r1,r2,r3 = np.random.rand(2**k,2**k),np.random.rand(2**k,2**k),np.random.rand(2**k,2**k)
#         u1,u2,u3 = [(1/np.sqrt(3))*sc.linalg.expm(-1j*(r1+Dagger(r1))/2),(1/np.sqrt(3))*sc.linalg.expm(-1j*(r2+Dagger(r2))/2),(1/np.sqrt(3))*sc.linalg.expm(-1j*(r3+Dagger(r3))/2)]
# #         print(np.linalg.norm(Dot(3*u1,Dagger(u1))-np.eye(2**k)))
#         krauslist = [u1,u2,u3]
#         krausdict[i]=(support,krauslist)
#     return (krausdict,Pi,Pjlist)

In [ ]:
# def random_test_PTelem_ij(dkraus,nkraus,jlistlength,n_qubits = 5):
#     (krausdict,Pi,Pjlist) = random_input_PTelem_ij(dkraus,nkraus,jlistlength,n_qubits)
#     return test_get_PTMelem_ij(krausdict,Pi,Pjlist,n_qubits)


In [ ]:
# %%time
# dkraus = 2
# nkraus = 10
# jlistlength = 100
# n_qubits = 5
# (krausdict,Pi,Pjlist) = random_input_PTelem_ij(dkraus,nkraus,jlistlength,n_qubits)
# get_PTMelem_ij(krausdict,Pi,Pjlist,n_qubits)
